In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
import os 
import tqdm
import glob
from PIL import Image
from tqdm import tqdm
from sklearn.utils import shuffle, resample
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from tensorflow import keras
from skimage.io import imread, imshow
from skimage.transform import resize
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.applications import ResNet50, DenseNet121,EfficientNetB1
from tensorflow.keras.utils import to_categorical
from keras import optimizers
from tensorflow.keras.optimizers import Adam, SGD
from keras.applications.nasnet import NASNetLarge, NASNetMobile
from keras.callbacks import Callback,ModelCheckpoint,ReduceLROnPlateau
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout,Input, UpSampling2D, GlobalAveragePooling2D,concatenate,Conv2DTranspose
from keras.models import Sequential,load_model,Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras.losses import categorical_crossentropy
import random
import keras.backend as K

In [ ]:
# import sys
# import platform
# print(sys.version)

In [ ]:
# import pkg_resources
# installed_packages = pkg_resources.working_set
# for package in installed_packages:
#     print(f"{package.key}=={package.version}")


In [ ]:
path = '/kaggle/input/severstal-steel-defect-detection'
train_df = pd.read_csv('/kaggle/input/severstal-steel-defect-detection/train.csv')
data_folder = '/kaggle/input/severstal-steel-defect-detection/train_images'
test_folder = '/kaggle/input/severstal-steel-defect-detection/test_images'
input_shape = (128,128)

In [ ]:
df1 = train_df[train_df.ClassId == 1]
df2 = train_df[train_df.ClassId == 2]
df3 = train_df[train_df.ClassId == 3]
df4 = train_df[train_df.ClassId == 4]
print(df1.shape, df2.shape, df3.shape, df4.shape)

In [ ]:
df3_undersampled = resample(df3, replace=False, n_samples=1500, random_state=42)  
df1_oversampled = resample(df1, replace=True, n_samples=1500, random_state=42)  
df2_oversampled = resample(df2, replace=True, n_samples=1500, random_state=42)  
df4_oversampled = resample(df4, replace=True, n_samples=1500, random_state=42)  

balanced_data = pd.concat([df1_oversampled, df2_oversampled, df3_undersampled, df4_oversampled])
balanced_data = balanced_data.sample(frac=1, random_state=42)
balanced_data

In [ ]:
def rle_decode(mask_rle: str = '', shape: tuple = (256, 1600)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')

def make_mask(balanced_data: pd.DataFrame, image_name: str='img.jpg', shape: tuple = (256, 1600)):
    encoded_masks = balanced_data.loc[balanced_data['ImageId'] == image_name, 'EncodedPixels']
    masks = np.zeros((shape[0], shape[1]), dtype=np.float32)
    for idx, label in enumerate(encoded_masks.values):
        if label is not np.nan:
            mask = rle_decode(label)
            masks[:, :] = mask
    return masks

def get_img(x, folder: str='Train_images'):
    image_path = os.path.join(data_folder, x)
    print(image_path)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
test_list = []
for file in os.listdir(test_folder):
    imgpath = os.path.join(test_folder, file)
    imgtest = cv2.imread(imgpath)
    imgtest = cv2.resize(imgtest, input_shape)
    test_list.append(imgtest)

In [ ]:
img_list = balanced_data["ImageId"].tolist()
label_list = balanced_data["ClassId"].tolist()
img_list[0:5], label_list[0:5]

In [ ]:
image_list = []
mask_list = []
for img in img_list:
    image = get_img(img)
    image = cv2.resize(image, input_shape)
    mask = make_mask(balanced_data, img)
    mask = cv2.resize(mask, input_shape)
    image_list.append(image)
    mask_list.append(mask)

In [ ]:
X = np.array(image_list)
y = np.array(mask_list)
encoded = LabelEncoder()
z=encoded.fit_transform(label_list)
z=to_categorical(z)
print(X.shape, y.shape, z.shape)

In [ ]:
X_train, X_val, y_train, y_val, z_train, z_val = train_test_split(X, y, z, test_size=0.2, shuffle=True)

In [ ]:
num_classes = 1
inputs = tf.keras.layers.Input((128, 128, 3))

c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
b1 = tf.keras.layers.BatchNormalization()(c1)
r1 = tf.keras.layers.ReLU()(b1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(r1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
b2 = tf.keras.layers.BatchNormalization()(c2)
r2 = tf.keras.layers.ReLU()(b2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(r2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
b3 = tf.keras.layers.BatchNormalization()(c3)
r3 = tf.keras.layers.ReLU()(b3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(r3)

c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
b4 = tf.keras.layers.BatchNormalization()(c4)
r4 = tf.keras.layers.ReLU()(b4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(r4)

c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
b5 = tf.keras.layers.BatchNormalization()(c5)
r5 = tf.keras.layers.ReLU()(b5)
c5 = tf.keras.layers.Dropout(0.3)(r5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
u6 = tf.keras.layers.BatchNormalization()(u6)
u6 = tf.keras.layers.ReLU()(u6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u6)
u7 = tf.keras.layers.concatenate([u7, c3])
u7 = tf.keras.layers.BatchNormalization()(u7)
u7 = tf.keras.layers.ReLU()(u7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(u7)
u8 = tf.keras.layers.concatenate([u8, c2])
u8 = tf.keras.layers.BatchNormalization()(u8)
u8 = tf.keras.layers.ReLU()(u8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(u8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
u9 = tf.keras.layers.BatchNormalization()(u9)
u9 = tf.keras.layers.ReLU()(u9)

outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='sigmoid')(u9)

In [ ]:
seg_model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
seg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 5,verbose = 1,factor = 0.50, min_lr = 1e-10)

In [ ]:
seg_history = seg_model.fit(X_train, y_train, epochs =45, batch_size=32, validation_data=(X_val, y_val), callbacks=[lrd])

In [ ]:
def display(display_list):
    plt.figure(figsize=(20, 20))
    title = ['Input image', 'True mask', 'Predicted mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')
    plt.show()

i = random.randint(0, len(X_val) - 1)
sample_image = X_val[i]
sample_mask = y_val[i]
prediction = seg_model.predict(sample_image[np.newaxis, ...])[0]
predicted_mask = (prediction > 0.5).astype(np.uint8)
display([sample_image, sample_mask, predicted_mask])

In [ ]:
prediction = seg_model.predict(sample_image[np.newaxis, ...])[0]
print(tf.reduce_max(prediction))

In [ ]:
# import pkg_resources
# installed_packages = pkg_resources.working_set
# for package in installed_packages:
#     print(f"{package.key}=={package.version}")

SAVE MODEL- 1

In [ ]:
# #Code to save model weights
# from keras.models import Model
# seg_model.save("//kaggle/working/segmentation_model.h5")

# """
# #to load model
# from keras.model import load_model
# loaded_model= load_model("segmentation_model.h5")
# """

In [ ]:
# from keras.models import Model
# import tensorflow as tf

# # Save the model in the TensorFlow SavedModel format
# tf.saved_model.save(seg_model, "//kaggle/working/segmentation_model.h5")


In [ ]:
seg_model.get_config()

In [ ]:
tf.keras.experimental.export_saved_model(seg_model, '//kaggle/working/segmentation_model.h5')

In [ ]:
seg_model.save('//kaggle/working/segmentation_model.tf')

In [ ]:
X_train=X_train/255
x_val=X_val/255

In [ ]:
!pip install efficientnet

In [ ]:
from tensorflow.keras.models import Model
from efficientnet.tfkeras import EfficientNetB1
class_model = EfficientNetB1(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
x_train = class_model.output
x_train = GlobalAveragePooling2D()(x_train)
x_train = Dense(512, activation='relu')(x_train)
x_train = Dropout(0.2)(x_train)
x_train = Dense(256, activation='relu')(x_train)
x_train = Dropout(0.2)(x_train)
x_train = Dense(128, activation='relu')(x_train)
x_train = Dropout(0.2)(x_train)
x_train = Dense(64, activation='relu')(x_train)
x_train = Dropout(0.2)(x_train)
predictions = Dense(4, activation='softmax')(x_train)
class_model = Model(inputs=class_model.input, outputs=predictions)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=5, min_lr=1e-8)
class_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 64
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)
train_generator = datagen.flow(X_train,z_train, batch_size=batch_size)

In [ ]:
class_history = class_model.fit(train_generator,epochs=35,validation_data=(x_val,z_val), batch_size=batch_size, verbose=1, callbacks=[lr_callback])

In [ ]:
# import pkg_resources
# installed_packages = pkg_resources.working_set
# for package in installed_packages:
#     print(f"{package.key}=={package.version}")

SAVE MODEL- 2

In [ ]:
# #Code to save model weights
# from keras.models import Model
# class_model.save("//kaggle/working/classification_model.h5")

# """
# #Code to load model weights
# from keras.models import load_model
# loaded_model=load_model("classification_model.h5")
# """

In [ ]:
class_model.get_config()

In [ ]:
# tf.keras.experimental.export_saved_model(class_model, '"//kaggle/working/classification_model.h5')

In [ ]:
class_model.save('//kaggle/working/classification_model.tf')

In [ ]:
# from keras.models import Model
# import tensorflow as tf

# # Save the model in the TensorFlow SavedModel format
# tf.saved_model.save(class_model, "//kaggle/working/classification_model.h5")

In [ ]:
i = random.randint(0, len(X_val) - 1)
sample_image = X_val[i]
sample_mask = y_val[i]
prediction = seg_model.predict(sample_image[np.newaxis, ...])[0]
predicted_mask = (prediction > 0.5).astype(np.uint8)
prediction_class = class_model.predict(sample_image[np.newaxis, ...])[0]
display([sample_image, sample_mask, predicted_mask])
print("Type defect: " ,np.argmax(prediction_class))

In [ ]:
def display_test(display_list):
    plt.figure(figsize=(15, 15))
    title = ['Input image', 'Predicted mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')
    plt.show()

X_test = np.array(test_list)
i = random.randint(0, len(X_test) - 1)
sample_image = X_test[i]
prediction = seg_model.predict(sample_image[np.newaxis, ...])[0]
predicted_mask = (prediction > 0.5).astype(np.uint8)
prediction_class = class_model.predict(sample_image[np.newaxis, ...])[0]
display_test([sample_image, predicted_mask])
print("Type defect: " ,np.argmax(prediction_class))

In [ ]:
def plot_training_history(history):
    
    train_accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    plt.subplot(1, 2, 1)  
    plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, label='Train')
    plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, label='Validation')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)  
    plt.plot(range(1, len(train_loss) + 1), train_loss, label='Train')
    plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_training_history(seg_history)

In [ ]:
plot_training_history(class_history)

In [ ]:
# !pip install streamlit

In [ ]:
# import streamlit as st
# import matplotlib.pyplot as plt
# import numpy as np
# import cv2
# from keras.models import load_model

# # Load the classification model
# classification_model = tf.saved_model.load("//kaggle/working/classification_model.h5")

# # Load the segmentation model
# segmentation_model = tf.saved_model.load("//kaggle/working/segmentation_model.h5")

# # Define a function for displaying test results
# def display_test(display_list):
#     plt.figure(figsize=(15, 15))
#     title = ['Input image', 'Predicted mask']
#     for i in range(len(display_list)):
#         plt.subplot(1, len(display_list), i + 1)
#         plt.title(title[i])
#         plt.imshow(display_list[i])
#         plt.axis('off')
#     st.pyplot()

# def main():
#     st.title("Defect Detection App")
#     st.sidebar.title("Upload an Image")

#     uploaded_image = st.sidebar.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

#     if uploaded_image is not None:
#         # Load the image using OpenCV
#         sample_image = cv2.cvtColor(cv2.imread(uploaded_image), cv2.COLOR_BGR2RGB)
#         sample_image = cv2.resize(sample_image, (128, 128))

#         # Perform inference
#         # Segmentation inference
#         prediction = segmentation_model.predict(sample_image[np.newaxis, ...])[0]
#         predicted_mask = (prediction > 0.5).astype(np.uint8)

#         # Classification inference
#         prediction_class = classification_model.predict(sample_image[np.newaxis, ...])[0]

#         # Display the input image and predicted mask
#         display_test([sample_image, predicted_mask])

#         # Display the defect type based on classification model
#         defect_type = np.argmax(prediction_class)
#         st.write("Type of defect:", defect_type)

# if __name__ == '__main__':
#     main()

    

    
    
    
    
    
    
    
# !streamlit run steel defect: segmen+classiify.ipynb


In [ ]:
# new_model = tf.keras.models.load_model('model.tf')

In [ ]:
#INFERENCE



import tensorflow as tf

# Load the classification model
classification_model = tf.keras.models.load_model("//kaggle/working/classification_model.tf")

# # Load the classification model
# from keras.models import load_model
# classification_model = load_model("//kaggle/working/classification_model.h5")

import tensorflow as tf

# Load the classification model
segmentation_model = tf.keras.models.load_model("//kaggle/working/segmentation_model.tf")

# # Load the segmentation model
# from keras.models import load_model
# segmentation_model = load_model("//kaggle/working/segmentation_model.h5")

# Define a function for displaying test results
def display_test(display_list):
    plt.figure(figsize=(15, 15))
    title = ['Input image', 'Predicted mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')
    plt.show()

# Input your own image file
image_path = "/kaggle/input/severstal-steel-defect-detection/test_images/0000f269f.jpg"

input_shape = (128,128)

# Load the image using OpenCV
import cv2
sample_image = cv2.imread(image_path)
sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
sample_image = cv2.resize(sample_image, input_shape)


# Perform inference
# Segmentation inference
prediction = segmentation_model.predict(sample_image[np.newaxis, ...])[0]
predicted_mask = (prediction > 0.5).astype(np.uint8)

# Classification inference
prediction_class = classification_model.predict(sample_image[np.newaxis, ...])[0]

display_test([sample_image, predicted_mask])
print("Type defect: ", np.argmax(prediction_class))


In [ ]:
# #INFERENCE

# # Load the classification model
# from keras.models import load_model
# classification_model = load_model("//kaggle/working/classification_model.h5")

# # Load the segmentation model
# from keras.models import load_model
# segmentation_model = load_model("//kaggle/working/segmentation_model.h5")

# # Define a function for displaying test results
# def display_test(display_list):
#     plt.figure(figsize=(15, 15))
#     title = ['Input image', 'Predicted mask']
#     for i in range(len(display_list)):
#         plt.subplot(1, len(display_list), i+1)
#         plt.title(title[i])
#         plt.imshow(display_list[i])
#         plt.axis('off')
#     plt.show()

# # Input your own image file
# image_path = "/kaggle/input/severstal-steel-defect-detection/test_images/002451917.jpg"

# input_shape = (128,128)

# # Load the image using OpenCV
# import cv2
# sample_image = cv2.imread(image_path)
# sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
# sample_image = cv2.resize(sample_image, input_shape)


# # Perform inference
# # Segmentation inference
# prediction = segmentation_model.predict(sample_image[np.newaxis, ...])[0]
# predicted_mask = (prediction > 0.5).astype(np.uint8)

# # Classification inference
# prediction_class = classification_model.predict(sample_image[np.newaxis, ...])[0]

# display_test([sample_image, predicted_mask])
# print("Type defect: ", np.argmax(prediction_class))
